In [1]:
import sys
from time import time
sys.path.append("../tools/")
from email_preprocess import preprocess

# Import scikit-learn metrics module for SVC
from sklearn import tree
# Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

/Users/yickminglee/.pyenv/versions/2.7.16/envs/minip/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
### features_train and features_test are the features for the training
### and testing datasets, respectively
### labels_train and labels_test are the corresponding item labels
features_train, features_test, labels_train, labels_test = preprocess()

no. of Chris training emails: 7936
no. of Sara training emails: 7884


In [3]:
features_train.shape

#len(features_train[0])
# 15,820 emails. some are Chris's and some are Sara's
# 3,785 features


(15820, 379)

In [4]:
import pickle
import cPickle
import numpy

from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif



def preprocess_pct(words_file = "../tools/word_data.pkl", authors_file="../tools/email_authors.pkl", percentile = 10):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features

        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions

        4 objects are returned:
            -- training/testing features
            -- training/testing labels

    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, "r")
    authors = pickle.load(authors_file_handler)
    authors_file_handler.close()

    words_file_handler = open(words_file, "r")
    word_data = cPickle.load(words_file_handler)
    words_file_handler.close()

    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)



    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=percentile)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print "no. of Chris training emails:", sum(labels_train)
    print "no. of Sara training emails:", len(labels_train)-sum(labels_train)
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test


In [5]:
pct = 10
print "Feature Selector Percentile: ", pct
features_train, features_test, labels_train, labels_test = preprocess_pct(percentile=pct)

# Create a SVM Classifier
clf = tree.DecisionTreeClassifier(min_samples_split=40)

# Train the model using the training sets
t0 = time()
clf.fit(features_train, labels_train)
training_time = round(time() - t0, 3)

# Predict the response for test dataset
t0 = time()
labels_test_pred = clf.predict(features_test)
prediction_time = round(time() - t0, 3)

# Model Accuracy, how often is the classifier correct?
accuracy = metrics.accuracy_score(labels_test, labels_test_pred)

print "Accuracy:", accuracy
print "Training time:", training_time, "s"
print "Prediction time:", prediction_time, "s"


Feature Selector Percentile:  no. of Chris training emails: 7936
no. of Sara training emails: 7884
Accuracy: 0.977815699659
Training time: 64.199 s
Prediction time: 0.058 s


In [6]:
pct = 1
print "Feature Selector Percentile: ", pct
features_train, features_test, labels_train, labels_test = preprocess_pct(percentile=pct)

# Create a SVM Classifier
clf = tree.DecisionTreeClassifier(min_samples_split=40)

# Train the model using the training sets
t0 = time()
clf.fit(features_train, labels_train)
training_time = round(time() - t0, 3)

# Predict the response for test dataset
t0 = time()
labels_test_pred = clf.predict(features_test)
prediction_time = round(time() - t0, 3)

# Model Accuracy, how often is the classifier correct?
accuracy = metrics.accuracy_score(labels_test, labels_test_pred)

print "Accuracy:", accuracy
print "Training time:", training_time, "s"
print "Prediction time:", prediction_time, "s"


Feature Selector Percentile:  no. of Chris training emails: 7936
no. of Sara training emails: 7884
Accuracy: 0.966439135381
Training time: 3.614 s
Prediction time: 0.001 s
